In [48]:
import random
import numpy as np
import copy
import itertools
from itertools import combinations
inf = np.inf
def convertiraenteros(lista):
    for indice,valor in enumerate(lista):
        lista[indice] = int(valor)
    return lista
def distancia(a, b):
    return ((a[1]-b[1])**(2)+(a[2]-b[2])**(2))**0.5
with open('a280.txt','r') as archivo:
    datos = archivo.read().split("\n")
    nombre = datos[0].split()[2]
    tipo = datos[1].split()[3]+datos[1].split()[4]+datos[1].split()[5]
    n = int(datos[2].split()[1])
    m = int(datos[3].split()[3])
    W = int(datos[4].split()[3])
    vmax = float(datos[5].split()[2])
    vmin = float(datos[6].split()[2])
    R = float(datos[7].split()[2])
    nodos = []
    d=[]
    pk=[]
    Wk = []
    disponibilidad = []
    i = 0
    for i in range(280): #TODO nodo (x,y)
        linea = datos[i+10].split()
        linea = convertiraenteros(linea)
        nodos.append(linea)
    for i in range(len(nodos)): #TODO valor de la matriz d
        renglon=[]
        for j in range(len(nodos)):
            renglon.append(distancia(nodos[i],nodos[j]))
        d.append(renglon)
    for i in range(279): #TODO pk valores
        pki = datos[i+291].split()[1]
        pk.append(pki)
        Wki = datos[i+291].split()[2]
        Wk.append(Wki)
        disponibilidadi = datos[i+291].split()[3]
        disponibilidadi = convertiraenteros([disponibilidadi])
        disponibilidad.append(disponibilidadi)
    pk = convertiraenteros(pk)
    Wk = convertiraenteros(Wk)
#disponibilidad = [[inf],[4,5],[1,2,3],[4]]
#TODO  modulo de inicializacion
x=[]
wc = 0
z = []
alfa = 1
for i in range(m):
    z.append(0)
vc=1
gz=0
def solinicial(x,d):
    nodoinicial = 1
    x.append(nodoinicial)
    posibles = []
    while len(x) < 280:
        minimo = inf    
        for indice, valor in enumerate(d[nodoinicial-1]):
            if indice + 1 not in x:
                if valor < minimo:
                    minimo = copy.deepcopy(valor)
        for indice, valor in enumerate(d[nodoinicial-1]):
            if valor == minimo:
                posibles.append(indice)
        nodosiguiente = random.choice(posibles) + 1
        if nodosiguiente not in x:
            x.append(nodosiguiente)
            nodoinicial = copy.deepcopy(nodosiguiente)
    return x

#TODO  funcion f
def f(z,x,Wk,vmin,vmax,W,vc,wc,d):
    wc=0
    ciudadesutilizadas = []
    for item in z:
        if item != 0:
            if item not in ciudadesutilizadas:
                ciudadesutilizadas.append(item)
    fxz = 0
    for i in x:
        for j in x:
            if x.index(i) - x.index(j) == -1:
                if i in ciudadesutilizadas:
                    for item in z:
                        if item == i:
                            wc+=Wk[z.index(item)]
                    vc = vmax - wc*(vmax-vmin)/W
                    fxz += d[i-1][j-1]/vc
                else:
                    fxz += d[i-1][j-1]/vc
            if x.index(i) == n-1 and x.index(j)==0:
                if i in ciudadesutilizadas:
                    for item in z:
                        if item == i:
                            wc+=Wk[z.index(item)]
                    vc = vmax - wc*(vmax-vmin)/W
                    fxz += d[i-1][j-1]/vc
                else:
                    fxz += d[i-1][j-1]/vc
    return fxz



#TODO  funcion gz
def g(m,gz,pk,z):
    itemsutilizados = []
    for i in range(m):  # ciclo para obtener los items utilizados
        if z[i] != 0:
            itemsutilizados.append(i)
    for i in itemsutilizados:
        gz+=pk[i]
    return gz


#TODO funcion Gxz
def Gxz(gz,fxz,R):
    return gz-R*fxz

#TODO  modulo de destruccion
# d_viajero es el numero de ciudades destruidas.
# d_item es el numero de items destruidos.




#TODO  se remueve una ciudad aleatoria
def eliminarciu(x):
    ciuelim = [] # ciudades eliminadas
    d_viajero = random.randint(2,279)
    for i in range(d_viajero):
        dviajero = random.randint(2,280)
        while(dviajero not in x):
            dviajero = random.randint(2,280)
        x.remove(dviajero)
        ciuelim.append(dviajero)
    return ciuelim

#TODO  se remueve un item aleatorio
def eliminaritem(z,wc):
    itemelim = [] # items eliminados
    d_item = random.randint(1,279)
    for i in range(d_item):
        ditem = random.randint(1,279)
        while(ditem in itemelim):
            ditem = random.randint(1,279)
        itemelim.append(ditem)
    
        for index, value in enumerate(z):
            if value == ditem:
                z[index] = 0
                wc -= Wk[index]
    return wc,itemelim

#TODO  modulo de contrucción ciudades
def consciu(ciuelim,x,d):
    while bool(ciuelim) == True:
        ultimaciudad = x[-1]
        #print(ultimaciudad)
        distanciamenor = inf
        for item in ciuelim:
            if d[ultimaciudad-1][item-1] < distanciamenor:
                distanciamenor = copy.deepcopy(d[ultimaciudad-1][item-1])
                nodoappend = item
                #print(nodoappend)
        x.append(nodoappend)
        ciuelim.remove(nodoappend)
        #print("ciuelim",ciuelim)
        #print("x",x)
    return x

def busqueda(x):
    for indice, valor in enumerate(x):
        if indice > 0 and indice < len(x)-1:
            y = copy.deepcopy(x)
            y[indice] = copy.deepcopy(x[indice + 1 ])
            y[indice + 1 ] = copy.deepcopy(x[indice])
            f1 = f(z,x,Wk,vmin,vmax,W,vc,wc,d)
            f2 = f(z,y,Wk,vmin,vmax,W,vc,wc,d)
            if f2 < f1:
              x = copy.deepcopy(y)
              break
    return x

def consciu2(x,ciuelim,d,alfa): #TODO consciu2
  while len(x) < n:
    listcostdisp = []
    ciudadesdisp = []
    nodoi = x[-1]
    for ciudad,i in enumerate(ciuelim):# costosdisponibles = costdisp
      costdisp = d[nodoi-1][i-1]
      listcostdisp.append(costdisp)
    if len(listcostdisp) >= 2: 
      max = np.max(listcostdisp)
      min = np.min(listcostdisp)
      rango = max - min
      cijmin = min+rango*alfa
      seleccionables = []
      for indice, valor in enumerate(d[nodoi-1]):
        if valor <= cijmin:
          if indice + 1 not in seleccionables and indice +1 not in x:
            seleccionables.append(indice+1)
      nodosiguiente = random.choice(seleccionables)
      x.append(nodosiguiente)
      ciuelim.remove(nodosiguiente)
    else:
      for item in range(1,n+1):
        if item not in x:
          x.append(item)
  return x

def empty(seq): 
    try: 
        return all(map(empty, seq)) 
    except TypeError: 
        return False 

#TODO construcción de items
#TODO disponibilidad = [[3],[3],[3],[2,4],[2]] #item[ciudad]
#TODO z=[0,3,3,0,0] item[ciudad]
def construcciónitems(disponibilidad,wc,W,z,Wk):
    dispoaux = copy.deepcopy(disponibilidad)
    while empty(dispoaux) == False:
        if wc == W:
            break
        for indice,valor in enumerate(z):
            try:
                if valor != 0:
                    dispoaux[indice].remove(valor)
            except:
                continue
        #inicialiacion de ciudad
        itemrandom = random.randint(1,m)
        while  len(dispoaux[itemrandom-1]) == 0: # ciclo para evitar la ciudad 1
            itemrandom = random.randint(1,m) # ciudad es la ciudad random seleccionada
        prueba =  wc + Wk[itemrandom-1]
        ciudad = random.choice(dispoaux[itemrandom-1])
        if prueba <= W and empty(dispoaux[itemrandom-1]) == False:
            z[itemrandom-1]=ciudad #asignación en el vector z
            wc += Wk[itemrandom-1] #actualización de wc
        dispoaux[itemrandom-1].remove(ciudad)
    return z,wc

#TODO main
contador =0
xinicial = solinicial(x,d)
zinicial, wcinicial = construcciónitems(disponibilidad,wc,W,z,Wk)
while contador < 1:
    finicial = f(zinicial,xinicial,Wk,vmin,vmax,W,vc,wcinicial,d)
    gzinicial = g(m,gz,pk,zinicial)
    Gxzinicial =  Gxz(gzinicial,finicial,R)
    ciuelim = eliminarciu(xinicial)
    wc2, itemelim = eliminaritem(zinicial,wcinicial)
    x2=consciu2(xinicial,ciuelim,d,alfa)
    x2 = busqueda(x2)
    #x2=consciu(ciuelim,xinicial,d)
    z2,wc2=construcciónitems(disponibilidad,wc2,W,zinicial,Wk)
    ffinal = f(z2,x2,Wk,vmin,vmax,W,vc,wc2,d)
    gzfinal = g(m,gz,pk,z2)
    Gxzfinal = Gxz(gzfinal,ffinal,R)
    # if Gxzfinal >= Gxzinicial:
    if ffinal <= finicial or gzfinal >= gzinicial:
    # if ffinal <= finicial and gzfinal >= gzinicial:
        contador+=1
        zinicial=z2
        xinicial=x2
        wcinicial=wc2
        print(contador)
print(zinicial,xinicial,Gxzinicial)

with open('resultados.txt','a') as resultados:
    texto = "\n"+"alfa: " + str(alfa) + " Gxz: " + str(Gxzinicial)  
    resultados.write(texto)

#FIXME probablemente si la construcción no es por vecinos mas cercanos no me quede en 45

KeyboardInterrupt: 

In [45]:
import random
import numpy as np
import copy
import itertools
from itertools import combinations
inf = np.inf
n=4 #TODO  numero de ciudades
m=5 #TODO  numero de items
vmax = 1 #TODO  velocidad maxima
vmin = 0.1 #TODO  velocidad minima
W=3 #TODO  peso maximo
Wk = [3,1,1,2,3] #TODO pesos de los items
pk = [100,40,40,20,30] #TODO ganancia por items
d = [[0,5,6,6],[5,0,5,6],[6,5,0,4],[6,6,4,0]] #TODO distancias 
#TODO inicializanción de las variables 
fxz = 0 
vc=1
wc=0
x=[1]
ciuelim = [2,4,3] 
z=[0,3,0,2,0]

def consciu2(x,ciuelim,d):
  alfa = 1
  while len(x) < n:
    listcostdisp = []
    ciudadesdisp = []
    nodoi = x[-1]
    for ciudad,i in enumerate(ciuelim):# costosdisponibles = costdisp
      costdisp = d[nodoi-1][i-1]
      listcostdisp.append(costdisp)
    print("costos disponibles",listcostdisp)
    if len(listcostdisp) >= 2: 
      max = np.max(listcostdisp)
      print("maximo: ",max)
      min = np.min(listcostdisp)
      print("minimo: ",min)
      rango = max - min
      print("rango: ",rango)
      cijmin = min+rango*alfa
      print("cijmin: ",cijmin)
      seleccionables = []
      for indice, valor in enumerate(d[nodoi-1]):
        if valor <= cijmin:
          if indice + 1 not in seleccionables and indice +1 not in x:
            seleccionables.append(indice+1)
      print("seleccionables",seleccionables)
      nodosiguiente = random.choice(seleccionables)
      print("nodosiguiente: ",nodosiguiente)
      x.append(nodosiguiente)
      print("x: ",x)
      ciuelim.remove(nodosiguiente)
    else:
      for item in range(1,n+1):
        if item not in x:
          x.append(item)
  return x

consciu2(x,ciuelim,d)

#TODO  funcion f
def f(z,x,Wk,vmin,vmax,W,vc,wc,d):
    wc=0
    ciudadesutilizadas = []
    for item in z:
        if item != 0:
            if item not in ciudadesutilizadas:
                ciudadesutilizadas.append(item)
    fxz = 0
    for i in x:
        for j in x:
            if x.index(i) - x.index(j) == -1:
                if i in ciudadesutilizadas:
                    for item in z:
                        if item == i:
                            wc+=Wk[z.index(item)]
                    vc = vmax - wc*(vmax-vmin)/W
                    fxz += d[i-1][j-1]/vc
                else:
                    fxz += d[i-1][j-1]/vc
            if x.index(i) == n-1 and x.index(j)==0:
                if i in ciudadesutilizadas:
                    for item in z:
                        if item == i:
                            wc+=Wk[z.index(item)]
                    vc = vmax - wc*(vmax-vmin)/W
                    fxz += d[i-1][j-1]/vc
                else:
                    fxz += d[i-1][j-1]/vc
    return fxz

#TODO busqueda local
def busqueda(x):
  temp = combinations(x,2)
  for item in temp:
      print(item)
      for indice, valor in enumerate(x):
          if indice > 0 and indice < len(x)-1:
              y = copy.deepcopy(x)
              y[indice] = copy.deepcopy(x[indice + 1 ])
              y[indice + 1 ] = copy.deepcopy(x[indice])
              f1 = f(z,x,Wk,vmin,vmax,W,vc,wc,d)
              f2 = f(z,y,Wk,vmin,vmax,W,vc,wc,d)
              if f2 < f1:
                x = copy.deepcopy(y)
                print(x)
                break
  return x

busqueda(x)

costos disponibles [5, 6, 6]
maximo:  6
minimo:  5
rango:  1
cijmin:  6
seleccionables [2, 3, 4]
nodosiguiente:  3
x:  [1, 3]
costos disponibles [5, 4]
maximo:  5
minimo:  4
rango:  1
cijmin:  5
seleccionables [2, 4]
nodosiguiente:  4
x:  [1, 3, 4]
costos disponibles [6]
(1, 3)
[1, 4, 3, 2]
(1, 4)
(1, 2)
(3, 4)
(3, 2)
(4, 2)


[1, 4, 3, 2]